1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래 걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM 질문과 같이 전달

1. 패키지 설치

In [ ]:
%pip install python-dotenv langchain langchain-upstage langchain-community langchain-text-splitters langchain-chroma pypdf

2. Knowledge Base 구성을 위한 데이터 생성

- RecursiveCharacterTextSplitter를 활용한 데이터 chunking
  - split 된 데이터 chunk를 Large Language Model(LLM)에게 전달하면 토큰 절약 가능
  - 비용 감소와 답변 생성시간 감소의 효과
  - LangChain에서 다양한 TextSplitter들을 제공
- chunk_size 는 split 된 chunk의 최대 크기
- chunk_overlap은 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = PyPDFLoader('./start-up_investor_relations.pdf')  # PDF 파일 경로를 지정
document_list = loader.load_and_split(text_splitter=text_splitter)

In [2]:
len(document_list)
document_list[1].page_content[:1000]

"4 5추천의 글\n추천의 글 | 카카오벤처스 정신아 대표\n이 책은 선배 창업가인 저자의 오랜 경험과 투자자와 스타트업의 \n인터뷰 등 현장의 목소리를 담은 책으로 그 어떤 이론보다 의미 있다\n고 본다. 또, 실제로 스타트업이 궁금한 부분을 물어가며 백서화시킨, \n스타트업에 대한 애정이 돋보이는 책이다. 스타트업이 투자자가 좋아\n할 모습을 추측하며 맞춰갈 때 안타까울 때가 많은데, 이 책을 통해 \n투자자를 잘 이해하게 되고, 스타트업의 실력이 투자자 앞에서 소통\n되어 투자자에게도 훌륭한 팀을 찾는 데 더 도움이 될 것이다. 스타트\n업을 위해 쓰인 이 책이 더 나아가 투자자와 스타트업의 연결고리가 \n될 것이라 믿으며, 본인의 지식, 경험과 정성을 담아 이 책을 쓰신 저\n자에게 감사드린다.추천의 글 | 스마트스터디 이승규 CFO\n투자 유치에 매진하던 2015년에 이 책을 읽었더라면, 나는 과거와 \n다른 선택을 적어도 세 가지는 했을 것 같다. 왜 투자가 필요한지, 지\n금 시점이 맞는지, 투자자는 어떤 생각을 하는지, 그리고 실제 투자 \n유치 과정이 어떻게 흘러가는지에 관해 '슬램 덩크'의 안경 선배처럼 \n옆에서 조곤조곤 설명해주는 멘토가 있으면 최상이겠지만, 없더라도 \n안도하고 바로 열어볼 수 있는 책이 드디어 나왔다.\n추천의 글 | 소프트뱅크벤처스아시아 이준표 대표\n투자 유치를 준비하는 창업가들이 반드시 읽어야 하는 필독서!\n누구나 처음으로 창업한 회사에 투자를 유치하는 과정은 막막하고 \n큰 두려움으로 다가온다. 이 책을 읽어보면 투자 유치를 위해서 꼭 알\n아야 하는 것들과 어디서 시작하면 좋을지, 또 그 과정을 어떻게 준비\n하면 좋을지를 자세히 알 수 있다. 이 책은 창업을 했거나 준비하는 \n창업자들에게 큰 도움이 될 길잡이가 될 것이다."

In [3]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = UpstageEmbeddings(
    model="embedding-query"
)

In [4]:
from langchain_chroma import Chroma

# 데이터를 처음 저장할 때
# database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma', persist_directory="./chroma")

# 이미 저장된 데이터를 사용할 때
database = Chroma(collection_name='chroma', persist_directory="./chroma", embedding_function=embedding)

3. 답변 생성을 위한 Retrieval

Chroma에 저장한 데이터를 유사도 검색(similarity_search())를 활용해서 가져옴

In [41]:
query = '스타트업 투자 자문을 위한 IR 자료는 어떤 내용이 포함되어야 하는가?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=7)

In [42]:
retrieved_docs

[Document(id='95b58207-3ceb-4263-b842-07985161a7c5', metadata={'creationdate': '2021-04-16T16:55:02+09:00', 'creator': 'Adobe InDesign 16.1 (Macintosh)', 'moddate': '2023-07-26T06:48:13+09:00', 'page': 91, 'page_label': '92', 'producer': 'Adobe PDF Library 15.0', 'source': './start-up_investor_relations.pdf', 'total_pages': 197, 'trapped': '/False'}, page_content='182 183H 팀 (Team)\n: 공동창업자들과 핵심 멤버들에 대한 설명  \n가끔 팀에 대한 정보를 빼고 IR 자료를 보내는 경우가 있는데, 이것\n은 큰 실수입니다. 그리고 멤버의 이름만 쓰는 것은 의미가 없고, 전\n공이나 어떤 경력이 있는지를 포함해야 합니다. 개인적으로 자주 하\n는 질문이 “당장은 아니더라도 미래에 어떤 드림팀이 이 사업을 가장 \n잘할 수 있을까요?”인데, 주요 멤버들의 이력을 통해 비즈니스 모델\n에 필요한 핵심 역량을 어느 정도 가졌는지를 잘 어필해야 합니다. 그\n리고 단순히 이름만 올린 외부 어드바이저 등은 너무 강조하지 않는 \n것이 좋죠.\n투자자는 공동창업자들을 비롯한 핵심 멤버들이 각자 어떤 삶을 살\n아왔고 어떻게 만났는지, 비전은 잘 공유되는지, 각자의 지분은 적절\n한지,34 그리고 얼마나 서로 끈끈한 관계인지도 궁금해하니 IR 피칭 \n때 보충 설명을 하면 좋습니다.\nI 재무 (Financial) \n: 3년 이내의 매출 계획과 예상되는 비용을 포함\n초기 스타트업의 경우 아직 가설 검증 단계이기에 미래 예측지표\n의 오차가 크기 마련입니다. 따라서 5년은 큰 의미가 없고 3년 이내\n34\u3000대표나 주요 멤버의 지분이 투자자의 예상보다 아주 낮으면 바람직

4. Augmentation을 위한 Prompt 활용

Retrieval된 데이터는 LangChain에서 제공하는 프롬프트("rlm/rag-prompt") 사용

In [7]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage(model='solar-pro')

In [8]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/Users/crazyatom/.pyenv/versions/start-up-bot/lib/python3.11/site-packages/langsmith/client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


5. 답변 생성

- RetrievalQA를 통해 LLM에 전달
  - RetrievalQA는 create_retrieval_chain으로 대체됨
  - 실제 ChatBot 구현 시 create_retrieval_chain으로 변경하는 과정을 볼 수 있음

In [43]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [44]:
# ai_message = qa_chain({"query": query})

# 업데이트된 LangChain 문법은 `.invoke()` 활용을 권장
ai_message = qa_chain.invoke({"query": query})

In [45]:
ai_message

{'query': '스타트업 투자 자문을 위한 IR 자료는 어떤 내용이 포함되어야 하는가?',
 'result': '스타트업 투자 자문을 위한 IR 자료는 팀 구성, 재무 계획, 비전을 포함해야 합니다. 팀 구성에서는 핵심 멤버들의 이력, 비전 공유, 지분, 관계 등을 나타내야 합니다. 재무 계획에서는 3년 이내의 매출 계획, 예상 비용, 투자금 활용 방안, 자금 소진 속도 등을 제시해야 합니다. 비전에서는 계획대로 잘될 경우 5년 후에 무엇을 성취할 것인지를 제시해야 합니다.'}